In [1]:
from IPython import parallel as p

In [2]:
cp = p.Client()

In [3]:
cells = np.load('./data/ucrb_cells.npy')

In [4]:
pths = pd.read_pickle('/Volumes/Users/Theo/projects/Budyko_vic/forcing_fluxes_filenames_lat_lon_index.df')
dates = pd.read_pickle('/Volumes/Users/Theo/projects/Budyko_vic/timecode.pcl')
fluxes_columns = ['y','m','d','ET','R','BF','sm1','sm2','sm3','SWE','Cs','Qs','Ql','Qg','NR','PEText','PETtrc','PETsrc']

def wateryear(index):
    year = index.year
    strt = pd.datetime(year,10,1) # start of the next water year+1
    
    if index<strt:
        wyear = year
    elif index>=strt: 
        wyear = year+1

    return wyear

def computesnowmelt(swe):
    swe = swe.as_matrix() # convert SWE to a matrix
    smelt = swe[0:-1]-swe[1:] # compute snowmelt for each day
    smelt = np.append(0,smelt)
    
    smelt[smelt<0]=0 # set accumulation events to 0
    
    return np.nanmean(smelt[smelt>0]) # compute the average snowmelt rate

In [5]:
%%px
pths = pd.read_pickle('/Volumes/Users/Theo/projects/Budyko_vic/forcing_fluxes_filenames_lat_lon_index.df')
dates = pd.read_pickle('/Volumes/Users/Theo/projects/Budyko_vic/timecode.pcl')
fluxes_columns = ['y','m','d','ET','R','BF','sm1','sm2','sm3','SWE','Cs','Qs','Ql','Qg','NR','PEText','PETtrc','PETsrc']

def wateryear(index):
    year = index.year
    strt = pd.datetime(year,10,1) # start of the next water year+1
    
    if index<strt:
        wyear = year
    elif index>=strt: 
        wyear = year+1

    return wyear

def computesnowmelt(swe):
    swe = swe.as_matrix() # convert SWE to a matrix
    smelt = swe[0:-1]-swe[1:] # compute snowmelt for each day
    smelt = np.append(0,smelt)
    
    smelt[smelt<0]=0 # set accumulation events to 0
    
    return np.nanmean(smelt[smelt>0]) # compute the average snowmelt rate
    

In [6]:
def ucr_met(cell):
    
    # get path from cell
    fl = pths.loc[pths.index == cell,'flux'].as_matrix()[0] # flux path

    flux = pd.read_table(fl, sep='\t', names = fluxes_columns) # load the data file
    flux.index = pd.DatetimeIndex(dates) 

    flux['wateryear'] = flux.index.map(wateryear) # compute water years
    
    # preallocate
    wys = []
    april1 = []
    peakswe = []
    smr = []
    
    # loop through each water year:
    for wy in np.unique(flux['wateryear'])[1:-1]: # iterate through the 2nd through 1 before the last water year
        # initialize lists for storage

        wys.append(wy)
        april1.append(flux.loc[str(wy)+'-04-01','SWE']) # compute april 1 SWE
        peakswe.append(pkswe) = flux.loc[flux.wateryear==wy,'SWE'].max() # compute peak SWE
        smr.append(computesnowmelt(flux.loc[flux.wateryear == wy,'SWE'])) # compute snowmelt
        
    n = len(wys)

    wys = np.reshape(wys,[1,n])[0,:]
    april1 = np.reshape(april1,[1,n])[0,:]
    peakswe = np.reshape(peakswe,[1,n])[0,:]
    smr = np.reshape(smr,[1,n])[0,:]

    outdf = pd.DataFrame()
    outdf['water_year'] = wys
    outdf['april_1_swe'] = april1
    outdf['peak_swe'] = peakswe
    outdf['snowmelt_rate'] = smr
    outdf.to_csv('/Volumes/Users/Theo/projects/Budyko_vic/snowmelt_gw/data/out/vic_'+str(int(cell))+'.csv', index=False)

In [7]:
view = cp.load_balanced_view()

In [8]:
res = view.map(ucr_met,cells)

In [58]:
res.progress

1729

In [59]:
res.progress

8563

### Create a list of lat, lon, and indexers for Brian Thomas

In [61]:
pths2 = pths[["lat","lon",'indexer']]

In [63]:
pths2.to_csv('./data/cell_centers_indexer.csv', index=False)